In [1]:
import numpy as np
from scipy.linalg import expm
from numpy import linalg as LA
from scipy import fft, ifft, fftpack
from numpy.linalg import matrix_power

## Kinetic Energy DVR code

In [2]:
def t_diag():
    
    """
    Return unscaled diagonal element of DVR kinetic energy.
    
    Output:
        v (float) - unscaled diagonal DVR matrix element
    """
    
    v = np.square(np.pi)/3.0
    return v


def t_off_diag(n, m):
    
    """
    Return unscaled off-diagonal element T[n,m] of DVR kinetic energy.
    
    Input:
        n,m (int) - indices of matrix elements
    
    Output:
        v (float) - unscaled off diagonal DVR matrix element
    """
    
    dif = n - m
    v = 2.0*np.power(-1.0,dif)/(np.square(dif))
    
    return v

def kin_dvr(D, mass, dx):
    
    """
    Return DVR kinetic energy matrix without any truncation.
    
    Input:
        D (int) - size of matrix
        mass (float) - mass of system
        dx (float) - spacing between grid points
        
    Output:
        kin (2D array) - DVR kineti energy matrix
        
    """
    
    #initialize kinetic energy matrix
    kin = np.zeros((D, D))
    
    for i in range(D):
        for j in range(D):
            if i == j:
                kin[i,i] = t_diag()
            else:
                kin[i,j] = t_off_diag(i,j)
     
    #scale by mass and dx
    kin = kin / (2.0 * mass * np.square(dx))
    return kin

In [3]:
def solve_width(nu):
    
    """
    Solve b = (d)/2^p such that b,p are integers, and b is odd, and return 2^p
    This function is used for Case 3b matrices.
    
    Input:
        nu (int) - even integer of Case 3b diagonal whose 1-sparse representation we want
        
    Output:
        width (int) - 2^p such that p solves the above equation
    """
    
    p = 1
    b = 0
    notfound = True
    
    while notfound:
        
        b = nu/(np.power(2, p))
        
        if (b+1)%2 == 0:
            
            notfound = False
        else:
            
            p = p + 1
    
    width = np.power(2, p)
    
    return width
    
def get_1_sparse(T, nu, D):
    
    """
    Return the 1-sparse decomposition of the nu-th diagonal of the DVR kinetic
    energy matrix. 
    
    Input:
        T (2d array) - DVR kinetic energy matrix
        nu (int) - diagonal whose 1-sparse representation we want.
        D (int) - dimension of T
        
    Output:
        [T_diag1, T_diag2] (array of 2d arrays) - 1-sprase decomposition of DVR kinetic energy matrix. 
                                                  The first 2d array contains sigma = 1, the second 
                                                  constains sigma = 2.
                                                 
    """
    
    "Extract diagonal d"
    diag_vec = np.diag(T, nu)
       
    #Case 1
    if nu == 0:
        T_diag = np.diag(diag_vec, nu)
        return [T_diag, np.zeros(T.shape)]
    
    #Case 2
    if nu > D/2:
        T_diag = np.diag(diag_vec, nu) + np.diag(diag_vec, -nu)
        return [T_diag, np.zeros(T.shape)]
    
    else:
        #Case 3a
        if (nu + 1) % 2 == 0:
            "nu is even"
            v1 = np.zeros(D - nu)
            v1[::2] += diag_vec[0]
            
            v2 = np.zeros(D - nu)
            v2[1::2] += diag_vec[0]
            
            T_diag1 = np.diag(v1, nu) + np.diag(v1, -nu)
            T_diag2 = np.diag(v2, nu) + np.diag(v2, -nu)
            return [T_diag1,T_diag2]

        #Case 3b
        else:
            "nu is odd"
            width = solve_width(nu)
            k=0
            v1 = np.zeros(D - nu)
            v2 = np.full(D - nu,diag_vec[0])
  
            while (2*k + 1)*width <= (D - nu):
                j = 2 * k
                v1[j*(width):(j+1)*(width)] = diag_vec[0]
                k += 1
            v2 = v2 - v1
            T_diag1 = np.diag(v1, nu) + np.diag(v1,- nu)
            T_diag2 = np.diag(v2, nu) + np.diag(v2,- nu) 
            
            return [T_diag1, T_diag2]

In [4]:
def one_sparse_kin(D, mass, dx, dt, l):
    
    """
    Return time evolution operator of the approximate DVR kinetic energy operator.
    
    Input:
        D (int) - size of matrix
        mass (float) - mass of system
        dx (float) - spacing between grid points
        dt (float) - simulation time
        l (int) - all diagonals with index greater than l are neglected
        
    Output:
        evol (2d array) - approximation to DVR kinetic energy operator truncated after the l-th diagonal
                          and using the 1-sparse decomposition
    """
    
    dvr_mat = kin_dvr(D, mass ,dx)
    
    #compute main diagonal
    m1, m2 = get_1_sparse(dvr_mat, 0 ,D)
    evol = expm(-1.j * dt * m1)

    #compute all other terms
    for j in range(1, l):
        m1, m2 = get_1_sparse(dvr_mat, j, D)
        prop1 = expm(-1.j * (dt) * m1)
        prop2 = expm(-1.j * (dt) * m2)
        prod = np.matmul(prop1, prop2)
        evol = np.matmul(evol, prod)

    return evol

## Symmetric Double Well Potential Energy code

In [5]:
def V_sdw(L, D, dx, mass, wb, V0):
    
    """
    Return symmetric double-well (sdw) potential evaluated over grid of length L and dimensions D.
    
    Input:
        L (float) - length of grid
        D (int) - number of grid points
        dx (float) - spacing between grid points 
        mass (float) - mass of system
        wb, V0 (float) - potential energy parameters
        
    Output:
        v_mat (2d array) - diagonal matrix containing sdw potential
    
    """
    
    v_diag = np.zeros(D, dtype ='complex')
    
    for i in range(D):
        x = -L/2.0 + (i * dx)
        v_diag[i] = -0.5 * mass * np.square(wb * x) + ((mass * mass * np.power(wb * x,4))/(16 * V0))
        
    v_mat = np.diag(v_diag)
    
    return v_mat

## Time Evolution Operators

In [6]:
def U_qft(D, mass, dx, dt, L, wb, V0):
    
    """
    Return time evolution under full Hamiltonian using QFT for kinetic energy
    
    Input:
        D (int) - number of grid points
        mass (float) - mass of system
        dx (float) - spacing between grid points 
        dt (float) - simulation time
        L (float) - length of grid
        wb, V0 (float) - potential energy parameters  
        
    Output:
        U_mat (2d mat) - matrix representation of time evolution operater using 2nd order Trotter and
                         QFT for kinetic energy term
    """
    
    #QFT parameters
    dx_rate = 1.0/dx
    pgrid = np.fft.fftfreq(D) * dx_rate * 2.0 * np.pi
    dp = pgrid[1]

    u_v = np.diag(np.exp(-1.j * 0.5 * dt * np.diag(V_sdw(L, D, dx, mass, wb, V0))))
    Ut = np.exp(- 1.j * 0.5 * np.square(pgrid) * dt / mass)
 
    U_mat = np.zeros((D,D),dtype='complex')
    
    for j in range(D):
        
        psiv_ft = fftpack.ifft(u_v[j,:])
        psiv_ft_u = fftpack.fft(Ut * psiv_ft)
        
        for k in range(D):
            U_mat[k,j] = np.dot(u_v[k,:], psiv_ft_u)

    return U_mat

In [7]:
def get_rte_sdw(D, mass, wb, V0, dx, dt, L, l, engine):
        
    """
    Return real-time evolution operator.
    
    Input:get_rte_sdw
        D (int) - dimension of system
        mass (float) - mass of system
        wb, V0 (float) - potential energy parameters  
        dx (float) - grid step size
        dt (float) - time step
        L (float) - grid length
        l (int) - all diagonals with index greater than l are neglected
        engine (string) - 'dvr' or 'qft'; specifies which kinetic energy popagation method is used

    Output:
        U_final (2d array) - requested time evolution operator
    """    
    
    if engine == 'dvr':
        U_t = one_sparse_kin(D, mass, dx, dt, l)
        vho = V_sdw(L, D, dx, mass, wb, V0)
        U_v = expm(-1.j * dt * vho / 2.0)
        U_final = np.matmul(U_v, np.matmul(U_t, U_v))
        return U_final
    
    elif engine == 'qft':   
        U_final = U_qft(D, mass, dx, dt, L, wb, V0)
        return U_final
    
    else :
        print("Invalid engine.")

def get_ite_sdw(D, mass, wb, V0, dx, dbeta, L, steps, l, engine):
    
    """
    Return imaginary-time evolution operator.
    
    Input:
        D (int) - dimension of system
        mass (float) - mass of system
        wb, V0 (float) - potential energy parameters  
        dx (float) - grid step size
        dbeta (float) - imaginary time simulation time
        L (float) - grid length
        step (int) - number of time steps taken
        l (int) - all diagonals with index greater than l are neglected
        engine (string) - 'dvr' or 'qft'; specifies which kinetic energy popagation method is used

    Output:
        ite_mat (2d array) - requested imaginary time evolution operator
    """    
    
    if engine == 'dvr':
    
        U_t = one_sparse_kin(D, mass, dx, -1.j * dbeta, l)    
        vho = V_sdw(L, D, dx, mass, wb, V0)
        U_v = expm(- dbeta * vho)
        U_dvr = np.matmul(U_t, U_v) 
        ite_mat = matrix_power(U_dvr, steps)
        return ite_mat
    
    elif engine == 'qft':
        ite_mat =  matrix_power(U_qft(D, mass, dx, -1.j * dbeta, L, wb, V0), steps)
        return ite_mat

    else:
        print("Invalide engine.")

    return ite_mat

## Correlation Function Code

In [8]:
def cxx_sdw(mass, wb, V0, L, D, steps, t, beta, l, real_engine, imag_engine):
    
    """
    Compute position-position TCF for symmetric double well (sdw) potential.
    
    Input:
        mass (float) - mass of system
        wb, V0 (float) - potential energy parameters  
        L (float) - grid length
        D (int) - dimension of systems
        steps (int)- number of imaginary time steps to take
        t (float) - simulation time
        beta (float) - imaginary time
        l (int)
        real_engine (string) - real-time evolution scheme; 'dvr' or 'qft'
        imag_engine (string) - imaginary-time evolution scheme; 'dvr' or 'qft'
        
    Output:
        cxx (1d array) - position-position TCF for sdw potential.  
    """
    
    #Initialize TFC array
    cxx = np.zeros(steps, dtype='complex')
    
    #Compute parameters
    dx = L/D
    dt = t/steps
    dbeta = beta/steps/2.0
    x = np.linspace(-L/2.0, L/2.0, D, endpoint=False)
    Xrow = np.ones((D, D)) * x
    Xcol = np.conjugate(Xrow)

    #Get imaginary time evolution operator
    ite = get_ite_sdw(D, mass, wb, V0, dx, dbeta, L, steps-1, l, imag_engine)
    ite_single = get_ite_sdw(D, mass, wb, V0, dx, dbeta, L, 1, l, imag_engine)
    
    X_ite = np.matmul(ite, Xrow * ite_single)

    #Get real time evolution operator
    rte = get_rte_sdw(D, mass, wb, V0, dx, dt, L, l, real_engine)
    rte_conj = np.transpose(np.conjugate(rte))

    #Compute step zero point
    cxx_0 = np.trace(matrix_power(X_ite, 2))
    cxx[0] = cxx_0

    #Compute step one point
    cxx_1 = np.trace(np.matmul(np.matmul(rte_conj, X_ite), np.matmul(rte, X_ite)))
    cxx[1] = cxx_1

    rte_step = rte
    rte_conj_step = rte_conj
    
    #Loop over remaining steps
    for j in range(steps-2):
        rte_step =  np.matmul(rte, rte_step)
        rte_conj_step =  np.matmul(rte_conj, rte_conj_step)
        
        cxx_step = np.trace(np.matmul(np.matmul(rte_conj_step, X_ite), np.matmul(rte_step, X_ite)))
        cxx[j + 2] = cxx_step
        
    return cxx

In [9]:
def cxx_exact_sdw(mass, wb, V0, L, D, t, beta, cutoff):
    
    """
    Compute exact position-position TCF using DVR diagonalization. There is no approximation to the
    DVR operator here.
    
    Input:
        mass (float) - mass of system
        wb, V0 (float) - potential energy parameters  
        L (float) - grid length
        D (int) - dimension of systems
        steps (int)- number of imaginary time steps to take
        t (float) - simulation time
        beta (float) - imaginary time
        cutoff (int) - eigenvalue cutoff; number of eigenvalues to retain in sum
        
    Output:
        corr (float) - position-position TCF for sdw potential at time t. The number of time steps 
        
    """
    
    #Define parameters
    dx = L/D
    kin_op = kin_dvr(D, mass, dx)
    vho = V_sdw(L, D, dx, mass, wb, V0)
    H_dvr = kin_op + vho
    eigenvalues, eigenvectors = LA.eig(H_dvr)
    sorted_inds = np.argsort(eigenvalues)
    
    eigenvalues = eigenvalues[sorted_inds]
    eigenvectors = eigenvectors[:, sorted_inds]
        
    x = np.linspace(-L/2.0, L/2.0, D, endpoint=False)
    Xrow = np.ones((D, D)) * x #constant across col
    Xcol = np.transpose(Xrow)
    
    M = np.matmul(np.transpose(np.conjugate(eigenvectors)), eigenvectors * Xcol) * np.square(dx)

    corr = 0
    for n in range(cutoff):
        e_n = eigenvalues[n]

        for m in range(cutoff):
            e_m = eigenvalues[m]

            exp_1 = np.exp(-(beta/2.0)*e_n)
            exp_2 = np.exp(-(beta/2.0)*e_m)
            exp_3 = np.exp(1.j * (e_n - e_m) * t)

            M_nm = M[n,m]
            M_mn = M[m,n]

            corr += M_nm * M_mn * exp_1 * exp_2 * exp_3
                
    return corr  